In [1]:
import keras
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.preprocessing import image
from keras.engine import Layer
from keras.applications.inception_resnet_v2 import preprocess_input
from keras.layers import Conv2D, UpSampling2D, InputLayer, Conv2DTranspose, Input, Reshape, merge, concatenate
from keras.layers import Activation, Dense, Dropout, Flatten
from keras.layers.normalization import BatchNormalization
from keras.callbacks import TensorBoard 
from keras.models import Sequential, Model
from keras.layers.core import RepeatVector, Permute
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from skimage.color import rgb2lab, lab2rgb, rgb2gray, gray2rgb
from skimage.transform import resize
from skimage.io import imsave
import numpy as np
import onnx
import keras2onnx
import os
import random
import tensorflow as tf
from keras.callbacks import EarlyStopping,ModelCheckpoint
import pandas as pd



Using TensorFlow backend.


In [2]:
category='Plant'
subcategory='pineapple'

def load_images_from_directory(directory_path):
    image_data = []
    for filename in os.listdir(directory_path):
        img = load_img(os.path.join(directory_path, filename))
        img_array = img_to_array(img)
        image_data.append(img_array)
    image_data = 1.0/255 * np.array(image_data, dtype=float)
    return image_data

train_images_path = 'images//Train//' + category + '_all//' + subcategory
valid_images_path = 'images//Valid//' + category + '_all//' + subcategory
Xtrain = load_images_from_directory(train_images_path)
Xvalid = load_images_from_directory(valid_images_path)


#Load weights
inception = InceptionResNetV2(weights='imagenet', include_top=True)
inception.graph = tf.get_default_graph()

In [3]:
embed_input = Input(shape=(1000,))

#Encoder
encoder_input = Input(shape=(256, 256, 1))
encoder_output = Conv2D(64, (3,3), activation='relu', padding='same', strides=2)(encoder_input)
encoder_output = Conv2D(128, (3,3), activation='relu', padding='same')(encoder_output)
encoder_output = Conv2D(128, (3,3), activation='relu', padding='same', strides=2)(encoder_output)
encoder_output = Conv2D(256, (3,3), activation='relu', padding='same')(encoder_output)
encoder_output = Conv2D(256, (3,3), activation='relu', padding='same', strides=2)(encoder_output)
encoder_output = Conv2D(512, (3,3), activation='relu', padding='same')(encoder_output)
encoder_output = Conv2D(512, (3,3), activation='relu', padding='same')(encoder_output)
encoder_output = Conv2D(256, (3,3), activation='relu', padding='same')(encoder_output)

#Fusion
fusion_output = RepeatVector(32 * 32)(embed_input) 
fusion_output = Reshape(([32, 32, 1000]))(fusion_output)
fusion_output = concatenate([encoder_output, fusion_output], axis=3) 
fusion_output = Conv2D(256, (1, 1), activation='relu', padding='same')(fusion_output) 

#Decoder
decoder_output = Conv2D(128, (3,3), activation='relu', padding='same')(fusion_output)
decoder_output = UpSampling2D((2, 2))(decoder_output)
decoder_output = Conv2D(64, (3,3), activation='relu', padding='same')(decoder_output)
decoder_output = UpSampling2D((2, 2))(decoder_output)
decoder_output = Conv2D(32, (3,3), activation='relu', padding='same')(decoder_output)
decoder_output = Conv2D(16, (3,3), activation='relu', padding='same')(decoder_output)
decoder_output = Conv2D(2, (3, 3), activation='tanh', padding='same')(decoder_output)
decoder_output = UpSampling2D((2, 2))(decoder_output)

model = Model(inputs=[encoder_input, embed_input], outputs=decoder_output)
#model = keras.models.load_model(os.path.join(os.getcwd(), 'models/models_h5/with_fusion/best_model_' + subcategory + '.h5'))

In [4]:
def create_inception_embedding(grayscaled_rgb):
    grayscaled_rgb_resized = []
    for i in grayscaled_rgb:
        i = resize(i, (299, 299, 3), mode='constant')
        grayscaled_rgb_resized.append(i)
    grayscaled_rgb_resized = np.array(grayscaled_rgb_resized)
    grayscaled_rgb_resized = preprocess_input(grayscaled_rgb_resized)
    with inception.graph.as_default():
        embed = inception.predict(grayscaled_rgb_resized)  
    return embed

# Image transformer
datagen = ImageDataGenerator(
        shear_range=0.4,
        zoom_range=0.4,
        rotation_range=40,
        horizontal_flip=True
)

# Generate training data
batch_size = 50

# EarlyStopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=125, restore_best_weights=False)

checkpoint = ModelCheckpoint('models/models_h5/with_fusion/best_model_' + subcategory + '_500.h5', monitor='val_loss', 
                             save_best_only=True)


def image_a_b_gen(X,batch_size):
    for batch in datagen.flow(X, batch_size=batch_size):
        grayscaled_rgb = gray2rgb(rgb2gray(batch))
        embed = create_inception_embedding(grayscaled_rgb)
        lab_batch = rgb2lab(batch)
        X_batch = lab_batch[:,:,:,0]
        X_batch = X_batch.reshape(X_batch.shape+(1,))
        Y_batch = lab_batch[:,:,:,1:] / 128
        yield [X_batch, create_inception_embedding(grayscaled_rgb)], Y_batch


# Train model with validation data
model.compile(optimizer='rmsprop', loss='mse')

hist = model.fit_generator(
        image_a_b_gen(Xtrain, batch_size),
        epochs=10000,
        steps_per_epoch=len(Xtrain) // batch_size,
        validation_data=image_a_b_gen(Xvalid, batch_size),
        validation_steps=len(Xvalid) // batch_size,
        callbacks=[early_stopping, checkpoint])  # Dodaj ModelCheckpoint do listy callbacks






Epoch 1/10000


D:\Programy\Anaconda\envs\Python_V\lib\site-packages\keras\utils\data_utils.py:709: UserWarning: An input could not be retrieved. It could be because a worker has died.We do not have any information on the lost sample.
  UserWarning)


10/10 [==============================] - 96s 10s/step - loss: 0.8097 - val_loss: 0.0195
Epoch 2/10000
10/10 [==============================] - 38s 4s/step - loss: 0.2226 - val_loss: 0.0277
Epoch 3/10000
10/10 [==============================] - 45s 4s/step - loss: 0.0265 - val_loss: 0.0205
Epoch 4/10000
10/10 [==============================] - 45s 5s/step - loss: 0.0242 - val_loss: 0.0223
Epoch 5/10000
10/10 [==============================] - 45s 5s/step - loss: 0.0196 - val_loss: 0.0188
Epoch 6/10000
10/10 [==============================] - 45s 4s/step - loss: 0.0187 - val_loss: 0.0202
Epoch 7/10000
10/10 [==============================] - 46s 5s/step - loss: 0.0185 - val_loss: 0.0191
Epoch 8/10000
10/10 [==============================] - 45s 4s/step - loss: 0.0186 - val_loss: 0.0353
Epoch 9/10000
10/10 [==============================] - 45s 5s/step - loss: 0.0208 - val_loss: 0.0219
Epoch 10/10000
10/10 [==============================] - 46s 5s/step - loss: 0.0192 - val_loss: 0.0197
Ep

10/10 [==============================] - 47s 5s/step - loss: 0.0175 - val_loss: 0.0172
Epoch 82/10000
10/10 [==============================] - 49s 5s/step - loss: 0.0170 - val_loss: 0.0173
Epoch 83/10000
10/10 [==============================] - 47s 5s/step - loss: 0.0169 - val_loss: 0.0172
Epoch 84/10000
10/10 [==============================] - 47s 5s/step - loss: 0.0171 - val_loss: 0.0177
Epoch 85/10000
10/10 [==============================] - 46s 5s/step - loss: 0.0164 - val_loss: 0.0176
Epoch 86/10000
10/10 [==============================] - 47s 5s/step - loss: 0.0166 - val_loss: 0.0170
Epoch 87/10000
10/10 [==============================] - 47s 5s/step - loss: 0.0166 - val_loss: 0.0174
Epoch 88/10000
10/10 [==============================] - 49s 5s/step - loss: 0.0171 - val_loss: 0.0173
Epoch 89/10000
10/10 [==============================] - 45s 5s/step - loss: 0.0171 - val_loss: 0.0175
Epoch 90/10000
10/10 [==============================] - 46s 5s/step - loss: 0.0167 - val_loss: 0.

Epoch 161/10000
10/10 [==============================] - 44s 4s/step - loss: 0.0158 - val_loss: 0.0185
Epoch 162/10000
10/10 [==============================] - 1317s 132s/step - loss: 0.0164 - val_loss: 0.0165
Epoch 163/10000
10/10 [==============================] - 44s 4s/step - loss: 0.0158 - val_loss: 0.0174
Epoch 164/10000
10/10 [==============================] - 45s 5s/step - loss: 0.0155 - val_loss: 0.0164
Epoch 165/10000
10/10 [==============================] - 47s 5s/step - loss: 0.0158 - val_loss: 0.0166
Epoch 166/10000
10/10 [==============================] - 47s 5s/step - loss: 0.0155 - val_loss: 0.0170
Epoch 167/10000
10/10 [==============================] - 47s 5s/step - loss: 0.0157 - val_loss: 0.0190
Epoch 168/10000
10/10 [==============================] - 48s 5s/step - loss: 0.0162 - val_loss: 0.0172
Epoch 169/10000
10/10 [==============================] - 46s 5s/step - loss: 0.0159 - val_loss: 0.0167
Epoch 170/10000
10/10 [==============================] - 46s 5s/step 

10/10 [==============================] - 48s 5s/step - loss: 0.0147 - val_loss: 0.0163
Epoch 241/10000
10/10 [==============================] - 48s 5s/step - loss: 0.0150 - val_loss: 0.0161
Epoch 242/10000
10/10 [==============================] - 49s 5s/step - loss: 0.0149 - val_loss: 0.0162
Epoch 243/10000
10/10 [==============================] - 47s 5s/step - loss: 0.0146 - val_loss: 0.0180
Epoch 244/10000
10/10 [==============================] - 46s 5s/step - loss: 0.0155 - val_loss: 0.0163
Epoch 245/10000
10/10 [==============================] - 45s 5s/step - loss: 0.0138 - val_loss: 0.0159
Epoch 246/10000
10/10 [==============================] - 45s 5s/step - loss: 0.0148 - val_loss: 0.0162
Epoch 247/10000
10/10 [==============================] - 46s 5s/step - loss: 0.0148 - val_loss: 0.0162
Epoch 248/10000
10/10 [==============================] - 45s 4s/step - loss: 0.0156 - val_loss: 0.0164
Epoch 249/10000
10/10 [==============================] - 45s 5s/step - loss: 0.0148 - val

10/10 [==============================] - 59s 6s/step - loss: 0.0142 - val_loss: 0.0163
Epoch 320/10000
10/10 [==============================] - 58s 6s/step - loss: 0.0135 - val_loss: 0.0164
Epoch 321/10000
10/10 [==============================] - 61s 6s/step - loss: 0.0145 - val_loss: 0.0169
Epoch 322/10000
10/10 [==============================] - 82s 8s/step - loss: 0.0134 - val_loss: 0.0162
Epoch 323/10000
10/10 [==============================] - 59s 6s/step - loss: 0.0144 - val_loss: 0.0162
Epoch 324/10000
10/10 [==============================] - 72s 7s/step - loss: 0.0143 - val_loss: 0.0159
Epoch 325/10000
10/10 [==============================] - 76s 8s/step - loss: 0.0138 - val_loss: 0.0172
Epoch 326/10000
10/10 [==============================] - 74s 7s/step - loss: 0.0142 - val_loss: 0.0182
Epoch 327/10000
10/10 [==============================] - 56s 6s/step - loss: 0.0143 - val_loss: 0.0159
Epoch 328/10000
10/10 [==============================] - 55s 5s/step - loss: 0.0129 - val

10/10 [==============================] - 53s 5s/step - loss: 0.0129 - val_loss: 0.0160
Epoch 399/10000
10/10 [==============================] - 53s 5s/step - loss: 0.0131 - val_loss: 0.0165
Epoch 400/10000
10/10 [==============================] - 53s 5s/step - loss: 0.0127 - val_loss: 0.0171
Epoch 401/10000
10/10 [==============================] - 54s 5s/step - loss: 0.0134 - val_loss: 0.0165
Epoch 402/10000
10/10 [==============================] - 68s 7s/step - loss: 0.0125 - val_loss: 0.0163
Epoch 403/10000
10/10 [==============================] - 64s 6s/step - loss: 0.0134 - val_loss: 0.0155
Epoch 404/10000
10/10 [==============================] - 64s 6s/step - loss: 0.0140 - val_loss: 0.0173
Epoch 405/10000
10/10 [==============================] - 67s 7s/step - loss: 0.0130 - val_loss: 0.0177
Epoch 406/10000
10/10 [==============================] - 67s 7s/step - loss: 0.0129 - val_loss: 0.0157
Epoch 407/10000
10/10 [==============================] - 50s 5s/step - loss: 0.0124 - val

In [5]:
model.save(os.path.join(os.getcwd(), 'models/models_h5/with_fusion/' + subcategory + '_500.h5'))

# Konwersja modelu na format ONNX
onnx_model = keras2onnx.convert_keras(model)

# Zapis modelu ONNX do pliku
onnx.save_model(onnx_model, os.path.join(os.getcwd(), 'models/models_onnx/with_fusion/' + subcategory + '_500.onnx'))


The ONNX operator number change on the optimization: 75 -> 39
The maximum opset needed by this model is only 11.


In [6]:
hist_df = pd.DataFrame(hist.history) 

hist_csv_file = 'history_' + subcategory + '.csv'
with open(hist_csv_file, mode='w') as f:
    hist_df.to_csv(f)